In [1]:
import os
import sys

PACKAGE_DIR = "/kaggle/src"
sys.path.append(PACKAGE_DIR)
sys.path.append(os.path.join(PACKAGE_DIR, "Penguin-ML-Library"))

In [2]:
import gc
import json
import warnings
from glob import glob

import numpy as np
import polars as pl
import yaml
from tqdm import tqdm

from utils import compute_ap, evaluate, load_list_bz2, read_bytes_in_range

In [5]:
files = glob("/kaggle/input/all-index-per-patent/data/*.json.bz2")
len(files)

12312871

In [10]:
for f in tqdm(files[: 50 * 10]):
    data = load_list_bz2(f)

100%|██████████| 500/500 [00:00<00:00, 1200.30it/s]


In [3]:
files = glob("/kaggle/input/preprocess-cands-ratio/cpc_token2patents/*.json.bz2")
len(files)

265889

In [4]:
count = 0
pbar = tqdm(files)
for f in pbar:
    data = load_list_bz2(f)
    count += len(data)
    pbar.set_postfix({"count": count})

  0%|          | 0/265889 [00:00<?, ?it/s]

  0%|          | 260/265889 [00:18<5:09:59, 14.28it/s, count=4393679]


KeyboardInterrupt: 

In [4]:
token2range = load_list_bz2("/kaggle/input/token-index/token2range.json.bz2")

In [5]:
with open("/kaggle/input/cpc-mapping/cpc2patents.json", "r") as f:
    cpc2patents = json.load(f)

In [9]:
import random

tokens = list(token2range.keys())
cpcs = list(cpc2patents.keys())

In [15]:
for _ in tqdm(range(10000)):
    token = random.choice(tokens)
    cpc = random.choice(cpcs)

    token_patents = set(
        read_bytes_in_range("/kaggle/input/token-index/index.txt", *token2range[token]).split(),
    )
    cpc_patents = set(cpc2patents[cpc])
    patents = token_patents & cpc_patents

100%|██████████| 10000/10000 [00:03<00:00, 3279.91it/s]


In [5]:
%cd /kaggle/working

/kaggle/working


In [6]:
import bz2
import json
import time

# サンプルのJSONデータを生成
data = [{"name": "Alice", "age": 30, "city": "New York"} for _ in range(1000000)]
json_file = "sample.json"
bz2_file = "sample.json.bz2"

# JSONファイルを保存
with open(json_file, "w") as f:
    json.dump(data, f)

# JSONファイルを圧縮して保存
with open(json_file, "rb") as f_in:
    with bz2.open(bz2_file, "wb") as f_out:
        f_out.writelines(f_in)

# 圧縮されていないJSONファイルの読み込み時間を測定
start_time = time.time()
with open(json_file, "r") as f:
    data_uncompressed = json.load(f)
uncompressed_time = time.time() - start_time

# 圧縮されたJSONファイルの読み込み時間を測定
start_time = time.time()
with bz2.open(bz2_file, "rt") as f:
    data_compressed = json.load(f)
compressed_time = time.time() - start_time

# 結果を表示
print(f"Uncompressed JSON load time: {uncompressed_time:.4f} seconds")
print(f"Compressed JSON load time: {compressed_time:.4f} seconds")

Uncompressed JSON load time: 0.3642 seconds
Compressed JSON load time: 0.7787 seconds


In [12]:
import time

import plyvel

# データベースの作成
db = plyvel.DB("./testdb/", create_if_missing=True)

# 挿入速度テスト
start_time = time.time()
for i in range(100000):
    db.put(f"key{i}".encode("utf-8"), f"value{i}".encode("utf-8"))
insert_duration = time.time() - start_time

# 検索速度テスト
start_time = time.time()
for i in range(100000):
    db.get(f"key{i}".encode("utf-8"))
search_duration = time.time() - start_time

db.close()

print(f"LevelDB - Insert duration: {insert_duration}s")
print(f"LevelDB - Search duration: {search_duration}s")

LevelDB - Insert duration: 0.12428855895996094s
LevelDB - Search duration: 0.06901955604553223s
